In [4]:
import pandas as pd
import numpy as np
import graphviz
from sklearn.model_selection import train_test_split
from sklearn import linear_model, metrics, tree
from IPython.display import SVG, Image, HTML

In [21]:
mush = pd.read_csv('D:/mushrooms.csv')

mush = mush.drop(mush.columns[5],axis='columns')

In [22]:
Y_names = np.unique(mush['class']).tolist()
y = mush['class'].values.tolist()

In [23]:
mush = mush.drop(columns=['class'])

In [24]:
col = mush.columns.tolist()
for i in range (len(col)):
    #dum = pd.get_dummies(mush[col[i]],prefix=col[i])
    #merged =  pd.concat([mush,dum],axis='columns')
    #mush = merged.drop([col[i],merged.columns[-1]],axis='columns')
    mush = pd.get_dummies(mush)

In [25]:
X=mush.values

In [26]:
y = pd.get_dummies(y)
y = y.drop(columns='e',axis='columns')
y = y.values

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X,y, shuffle=True, stratify=y, test_size=0.2)

In [28]:
log_full = linear_model.LogisticRegression()
log_full.fit(X_train, y_train)

C:\Users\Leonid\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Leonid\Anaconda3\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [29]:
full_predict = log_full.predict(X_test)
print("Точность логистической регрессии:", metrics.accuracy_score(y_test, full_predict))

Точность логистической регрессии: 1.0


In [30]:
dt_pruned = tree.DecisionTreeClassifier()
dt_pruned.fit(X_train, y_train)
dt_pred = dt_pruned.predict(X_test)
print('Точность дерева:', metrics.accuracy_score(y_test, dt_pred))

Точность дерева: 1.0


In [31]:
col = mush.columns.tolist()
dot = graphviz.Source(tree.export_graphviz(dt_pruned, feature_names=col, class_names=Y_names))
svg_bytes = dot.pipe(format='svg')
no_wrap_div = '<div style="width:80% ;height:80% ;">{}</div>'
HTML(no_wrap_div.format(svg_bytes.decode()))

In [16]:
dot.format = 'png'
dot.render('dtree_render',view=True)

'dtree_render.png'

In [20]:
importants = {}
for i, score in enumerate(dt_pruned.feature_importances_):
    if dt_pruned.feature_importances_[i]>0:
        importants[col[i]]=dt_pruned.feature_importances_[i]
from collections import OrderedDict
importants = OrderedDict(sorted(importants.items(), key=lambda x: x[1],reverse=True))
importants        

OrderedDict([('stalk-surface-above-ring_k', 0.34367195549754204),
             ('gill-size_n', 0.310899585548384),
             ('spore-print-color_h', 0.12815548616247444),
             ('ring-type_p', 0.06765666146528934),
             ('spore-print-color_r', 0.03388311162073978),
             ('cap-surface_f', 0.03110061234115299),
             ('stalk-surface-below-ring_f', 0.02609062651711936),
             ('bruises_t', 0.014579384155361159),
             ('habitat_d', 0.01406957200152763),
             ('gill-spacing_w', 0.013585067475089274),
             ('stalk-shape_t', 0.010658782493673184),
             ('stalk-color-below-ring_n', 0.00564915472164679)])